In [9]:
import os
from google.cloud import bigquery

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"/home/imdatsing/airflow/dags/__pycache__/seventh-jet-424513-h5-a426c383f9d7.json"

def read_sql_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

# Function to execute a SQL query
def execute_query(file_path):
    client = bigquery.Client(location="asia-southeast1")
    sql_query = read_sql_file(file_path)
    query_job = client.query(sql_query)
    query_job.result()
    print(f"Query from {file_path} executed successfully.")
    
execute_query('/home/imdatsing/airflow/dags/__pycache__/BigQuery/Fact_RFM.sql')

Query from /home/imdatsing/airflow/dags/__pycache__/BigQuery/Fact_RFM.sql executed successfully.


In [4]:
def read_sql_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()


read_sql_file('/home/imdatsing/airflow/dags/__pycache__/BigQuery/Fact_RFM.sql')

"ALTER TABLE `seventh-jet-424513-h5.OLAP.Fact`\nADD COLUMN IF NOT EXISTS recency_score INT64,\nADD COLUMN IF NOT EXISTS frequency_score INT64,\nADD COLUMN IF NOT EXISTS monetary_score INT64,\nADD COLUMN IF NOT EXISTS RFM_segment STRING;\n\nCREATE OR REPLACE TABLE `seventh-jet-424513-h5.OLAP.LastOrderDate` AS\nSELECT\n  MAX(orderdate) AS last_order_date\nFROM\n  `seventh-jet-424513-h5.OLAP.Fact`;\n\nCREATE OR REPLACE TABLE `seventh-jet-424513-h5.OLAP.Fact_RFM` AS\nWITH last_date AS (\n  SELECT last_order_date FROM `seventh-jet-424513-h5.OLAP.LastOrderDate`\n),\nrfm_values AS (\n  SELECT\n    customerid,\n    MAX(orderdate) AS last_order_date,\n    COUNT(customerid) AS frequency,\n    SUM(totalordertoUSD) AS monetary\n  FROM\n    `seventh-jet-424513-h5.OLAP.Fact`\n  GROUP BY\n    customerid\n),\nrfm_scores AS (\n  SELECT\n    customerid,\n    DATE_DIFF((SELECT last_order_date FROM last_date), last_order_date, DAY) AS recency,\n    frequency,\n    monetary\n  FROM\n    rfm_values\n),\nrfm

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

def train_linear_regression_model():
    # Initialize BigQuery client
    client = bigquery.Client(location="asia-southeast1")
    
    # Define the query to retrieve data from the Fact table
    query = """
    SELECT * FROM `seventh-jet-424513-h5.OLAP.Fact`
    """
    
    # Execute the query and convert the result to a Pandas DataFrame
    df = client.query(query).to_dataframe()
    
    # Data preprocessing
    df['orderdate'] = pd.to_datetime(df['orderdate'])  # Convert orderdate to datetime
    df['order_year'] = df['orderdate'].dt.year         # Extract year from orderdate
    df = df.drop(['orderdate', 'salesorderid'], axis=1)  # Drop orderdate and salesorderid columns
    
    # Fill missing values with 0
    df = df.fillna(0)
    
    # Select features (X) and target (y)
    X = df.drop('totaldue', axis=1)  # Features: all columns except totaldue
    y = df['totaldue']               # Target: totaldue

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    predictions = model.predict(X_test)
    
    # Evaluate the model using Mean Squared Error (MSE)
    mse = mean_squared_error(y_test, predictions)
    
    # Create a DataFrame with the test set and predictions
    results_df = X_test.copy()
    results_df['actual_totaldue'] = y_test
    results_df['predicted_totaldue'] = predictions
    
    # Evaluate the model using Mean Squared Error (MSE)
    mse = mean_squared_error(y_test, predictions)
    print(f"Mean Squared Error: {mse}")
    
    # Push the results to BigQuery
    table_id = "seventh-jet-424513-h5.Machine_Learning_Results.LinearRegressionResults"
    job = client.load_table_from_dataframe(results_df, table_id)
    job.result()  # Wait for the job to complete
    print(f"Results successfully loaded to BigQuery table {table_id}")
train_model()

Mean Squared Error: 1.4008638617217284e-22
Results successfully loaded to BigQuery table seventh-jet-424513-h5.Machine_Learning_Results.LinearRegressionResults


In [19]:
from google.cloud import bigquery
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Set the Google Cloud credentials environment variable
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"/home/imdatsing/airflow/dags/__pycache__/seventh-jet-424513-h5-a426c383f9d7.json"

def customer_clustering():
    # Initialize BigQuery client
    client = bigquery.Client()
    
    # Define the query to retrieve customer data
    query = """
    SELECT * FROM `your_dataset.your_customer_table`
    """
    
    # Execute the query and convert the result to a Pandas DataFrame
    df = client.query(query).to_dataframe()
    
    # Data preprocessing
    # (Add your preprocessing steps here if needed)
    
    # Feature selection for clustering
    features = df.drop(['customer_id'], axis=1)  # Exclude customer_id column from features
    
    # Standardize the features
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    
    # Perform PCA to reduce dimensionality for better clustering visualization (optional)
    pca = PCA(n_components=2)
    features_pca = pca.fit_transform(features_scaled)
    
    # Perform K-Means clustering
    kmeans = KMeans(n_clusters=5, random_state=42)  # Assume we want 5 clusters
    df['cluster'] = kmeans.fit_predict(features_scaled)
    
    # Push the results to BigQuery
    table_id = "your_dataset.customer_clusters"
    job = client.load_table_from_dataframe(df, table_id)
    job.result()  # Wait for the job to complete
    print(f"Customer clustering results successfully loaded to BigQuery table {table_id}")

train_clustering_model()


Clustering results successfully loaded to BigQuery table seventh-jet-424513-h5.Machine_Learning_Results.ClusteringResults


In [56]:
import time
from datetime import datetime
import pandas as pd          # pip install pandas
import pypyodbc as odbc      # pip install pypyodbc
from google.cloud import bigquery  # pip install google-cloud-bigquery
import io                    # Import the io module
import os
from google.oauth2 import service_account
from sqlserver import SQLServer

class SQLServer:
    DRIVER_NAME = 'ODBC Driver 17 for SQL Server'

    def __init__(self, server_name, database_name, username, password):
        self.server_name = server_name
        self.database_name = database_name
        self.username = username
        self.password = password
        self.conn = None

    def connect_to_sql_server(self):
        try:
            self.conn = odbc.connect(self._connection_string())
            print('Connected')
            return self.conn
        except Exception as e:
            print(e)
            return None

    def _connection_string(self):
        conn_string = f"""
            DRIVER={{{self.DRIVER_NAME}}};
            SERVER={self.server_name};
            DATABASE={self.database_name};
            UID={self.username};
            PWD={self.password};
        """
        return conn_string

    def query(self, sql_statement):
        if self.conn is None:
            print('Connection is absent')
            return None, None
        try:
            cursor = self.conn.cursor()
            cursor.execute(sql_statement)
            data = cursor.fetchall()
            columns = [col[0] for col in cursor.description]
            return columns, data
        except Exception as e:
            print(e)
            return None, None

    def cursor(self):
        if not self.check_connection():
            print('Connection is absent')
            return None
        return self.conn.cursor()

    def check_connection(self):
        return self.conn is not None

# Step 1. Connect to SQL Server database
SERVER_NAME = '172.30.114.39'
DATABASE_NAME = 'AdventureWorks2022'
USERNAME = 'sa'
PASSWORD = 'Imdatsing15122003.'

# Create a connection string
sql_server_instance = SQLServer(SERVER_NAME, DATABASE_NAME,USERNAME,PASSWORD)
sql_server_instance.connect_to_sql_server()

# Step 2. Set up Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"/home/imdatsing/airflow/dags/__pycache__/seventh-jet-424513-h5-a426c383f9d7.json"


('HYT00', '[HYT00] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired')


In [52]:
def load_data_to_bigquery(sql_query, table_id):
    columns, records = sql_server_instance.query(sql_query)
    df = pd.DataFrame(records, columns=columns)
    df["Created_dt"] = datetime.now()

    # Define table schema
    client = bigquery.Client(location="asia-southeast1")

    # Configure job
    job_config = bigquery.LoadJobConfig(
        autodetect=True,
        write_disposition='WRITE_TRUNCATE'
    )

    # Convert the DataFrame to a CSV string
    csv_data = df.to_csv(index=False)

    # Load data to BigQuery from CSV string
    job = client.load_table_from_file(
        file_obj=io.StringIO(csv_data),
        destination=table_id,
        job_config=job_config
    )

    # Wait for the load job to complete
    job.result()
    print(f"Data loaded successfully to BigQuery table {table_id}")

In [53]:
# Define the SQL queries and corresponding BigQuery table IDs
tasks = [
    {"sql_query": "SELECT * FROM Sales.Customer", "table_id": "seventh-jet-424513-h5.STAGING.Customer"},

    {"sql_query": "SELECT * FROM Sales.CurrencyRate", "table_id": "seventh-jet-424513-h5.STAGING.CurrencyRate"},

    {"sql_query": "SELECT * FROM Sales.Currency", "table_id": "seventh-jet-424513-h5.STAGING.Currency"},

    {"sql_query": "SELECT * FROM Sales.SalesOrderHeader", "table_id": "seventh-jet-424513-h5.STAGING.SalesOrderHeader"},

    {"sql_query": "SELECT * FROM Sales.SalesOrderDetail", "table_id": "seventh-jet-424513-h5.STAGING.SalesOrderDetail"},

    {"sql_query": "SELECT * FROM Production.Product", "table_id": "seventh-jet-424513-h5.STAGING.Product"},

    {"sql_query": "SELECT * FROM Production.ProductSubcategory", "table_id": "seventh-jet-424513-h5.STAGING.ProductSubcategory"},

    {"sql_query": "SELECT * FROM Production.ProductCategory", "table_id": "seventh-jet-424513-h5.STAGING.ProductCategory"},

    {"sql_query": "SELECT * FROM Person.StateProvince", "table_id": "seventh-jet-424513-h5.STAGING.StateProvince"},

    {"sql_query": "SELECT * FROM Person.CountryRegion", "table_id": "seventh-jet-424513-h5.STAGING.CountryRegion"},

    {"sql_query": "SELECT * FROM Sales.SalesOrderHeaderSalesReason", "table_id": "seventh-jet-424513-h5.STAGING.SalesOrderHeaderSalesReason"},

    {"sql_query": "SELECT * FROM Sales.SalesReason", "table_id": "seventh-jet-424513-h5.STAGING.SalesReason"},
]

# Run the load process for each task
for task in tasks:
    load_data_to_bigquery(task["sql_query"], task["table_id"])

Data loaded successfully to BigQuery table seventh-jet-424513-h5.STAGING.Customer
('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is insufficient system memory in resource pool 'internal' to run this query.")
Data loaded successfully to BigQuery table seventh-jet-424513-h5.STAGING.CurrencyRate
('HY000', '[HY000] [Microsoft][ODBC Driver 17 for SQL Server]Unspecified error occurred on SQL Server. Connection may have been terminated by the server.')


Exception ignored in: <function Cursor.__del__ at 0x7f021a58feb0>
Traceback (most recent call last):
  File "/home/imdatsing/.local/lib/python3.10/site-packages/pypyodbc.py", line 2389, in __del__
    self.close()
  File "/home/imdatsing/.local/lib/python3.10/site-packages/pypyodbc.py", line 2372, in close
    check_success(self, ret)
  File "/home/imdatsing/.local/lib/python3.10/site-packages/pypyodbc.py", line 1007, in check_success
    ctrl_err(SQL_HANDLE_STMT, ODBC_obj.stmt_h, ret, ODBC_obj.ansi)
  File "/home/imdatsing/.local/lib/python3.10/site-packages/pypyodbc.py", line 987, in ctrl_err
    raise DatabaseError(state,err_text)
pypyodbc.DatabaseError: ('08S01', '[08S01] [Microsoft][ODBC Driver 17 for SQL Server]Communication link failure')


Data loaded successfully to BigQuery table seventh-jet-424513-h5.STAGING.Currency
('08S01', '[08S01] [Microsoft][ODBC Driver 17 for SQL Server]TCP Provider: Error code 0x2746')
Data loaded successfully to BigQuery table seventh-jet-424513-h5.STAGING.SalesOrderHeader
('08S01', '[08S01] [Microsoft][ODBC Driver 17 for SQL Server]Communication link failure')
Data loaded successfully to BigQuery table seventh-jet-424513-h5.STAGING.SalesOrderDetail
('08S01', '[08S01] [Microsoft][ODBC Driver 17 for SQL Server]Communication link failure')
Data loaded successfully to BigQuery table seventh-jet-424513-h5.STAGING.Product
('08S01', '[08S01] [Microsoft][ODBC Driver 17 for SQL Server]Communication link failure')
Data loaded successfully to BigQuery table seventh-jet-424513-h5.STAGING.ProductSubcategory
('08S01', '[08S01] [Microsoft][ODBC Driver 17 for SQL Server]Communication link failure')
Data loaded successfully to BigQuery table seventh-jet-424513-h5.STAGING.ProductCategory
('08S01', '[08S01] [M

In [47]:
# Step 2. Query the records from SQL Server
sql_statement = """
SELECT 
AddressID,
AddressLine1,
AddressLine2,
City,
StateProvinceID

FROM Person.Address 
"""

columns, records = sql_server_instance.query(sql_statement)
Address = pd.DataFrame(records, columns=columns)
Address["Created_dt"] = datetime.now()
Address["addressline1"] = Address["addressline1"].astype(str)
Address["addressline2"] = Address["addressline1"].astype(str)

# Step 3. Define table schema
client = bigquery.Client(location="asia-southeast1")

# If you don't want to specify column schema, set autodetect to True
job_config = bigquery.LoadJobConfig(
    autodetect=True,
    #schema=[
        #bigquery.SchemaField("orderdate", 'TIMESTAMP'),
        #bigquery.SchemaField("duedate", 'TIMESTAMP'),
        #bigquery.SchemaField("shipdate", 'TIMESTAMP'),
    #],
    # WRITE_APPEND, WRITE_EMPTY
    write_disposition='WRITE_TRUNCATE'
)

# Step 4. Load data to BigQuery
table_id = 'seventh-jet-424513-h5.STAGING.Address'  # Replace with your project and dataset ID

# Convert the DataFrame to a CSV string
csv_data = Address.to_csv(index=False)

# Load data to BigQuery from CSV string
job = client.load_table_from_file(
    file_obj=io.StringIO(csv_data),
    destination=table_id,
    job_config=job_config
)

# Wait for the load job to complete
job.result()

print("Data loaded successfully to BigQuery")

BadRequest: 400 PUT https://bigquery.googleapis.com/upload/bigquery/v2/projects/seventh-jet-424513-h5/jobs?uploadType=resumable&upload_id=ABPtcPqlqFOxz4kQu5zDLqI5FtCXikKAWWLj98TR0NSZhSpUHOtiz09zP9CgFjhiJ1zL0MktbXwG7x524S0uUyS3OC-k8VrasSjAdyqyXMAsC9ui: Invalid request.  There were 1624989 byte(s) (or more) in the request body.  There should have been 1623125 byte(s) (starting at offset 0 and ending at offset 1623124) according to the Content-Range header.